# Graph Neural Networks
## What are Graph Neural Networks (GNNs)?

In [1]:
#import the basics
import random
from sched import scheduler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch_geometric
import torch.nn as nn
from torch import optim, Tensor
import torch.nn.functional as F
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.datasets import AmazonBook, MovieLens
from torch_geometric.transforms import Compose, ToDevice, ToUndirected
from torch_geometric.data import Data
from torch_geometric.typing import Adj
from torch_sparse import SparseTensor, matmul
from torch_geometric.utils import train_test_split_edges
%matplotlib inline

In [2]:
torch_geometric.seed_everything(1234)
torch_geometric.__version__

'2.6.1'

In [3]:
# Let's verify what device we are working with
device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu


Graph Neural Networks are a type of "geometric deep learning" models that use pairwise message passing. They typically have an architecture consisting of 3 types of layers. From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network):
1. Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via **pairwise message passing between graph nodes**. Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
2. Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling, and self-attention pooling.
3. Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output. Examples include element-wise sum, mean or maximum.

## Attributes
- [T]he preprocessing step first
“squashes” the graph structured data into a vector of reals and
then deals with the preprocessed data using a list-based data
processing technique. However, important information, e.g., the
topological dependency of information on each node may be
lost during the preprocessing stage and the final result may depend, in an unpredictable manner, on the details of the preprocessing algorith [1] **GNNS preserve the structure of the graph it is based on.**
- It will be shown that the GNN
is an extension of both recursive neural networks and random
walk models and that it retains their characteristics. The model
extends recursive neural networks since it can process a more
general class of graphs including cyclic, directed, and undirected graphs, and it can deal with node-focused applications
without any preprocessing steps. The approach extends random
walk theory by the introduction of a learning algorithm and by
enlarging the class of processes that can be modeled. [1]
- Weights are shared across layer structures

### What is message passing?
From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers):
<br>
![img](./img/notebook/messagePassing.png)

## Computation Graph
"The neighbour of a node defines its computation graph" - @12:34 https://www.youtube.com/watch?v=JtDgmmQ60x8&ab_channel=AntonioLonga



In [4]:
saved_tensors = torch.load('sparse_tensors.pt')
train_sparse = saved_tensors['train_sparse']
val_sparse = saved_tensors['val_sparse']
test_sparse = saved_tensors['test_sparse']

sparse_sizes = train_sparse.sparse_sizes()
num_users = num_items = sparse_sizes[0] // 2
print(f"Number of users/items: {num_users}")
print(f"Size of sparse matrix: {sparse_sizes}")

train_edge_index = torch.stack([train_sparse.storage.row(), train_sparse.storage.col()])
val_edge_index = torch.stack([val_sparse.storage.row(), val_sparse.storage.col()])

train_edge_index = train_edge_index.to(device)
val_edge_index = val_edge_index.to(device)
train_sparse = train_sparse.to(device)
val_sparse = val_sparse.to(device)

Number of users/items: 5167
Size of sparse matrix: (10334, 10334)


C:\Users\Rob\AppData\Local\Temp\ipykernel_12748\3870268926.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_tensors = torch.load('sparse_tensors.pt')


In [5]:
print(train_sparse.storage.row()[100:200])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3])


In [6]:
print(train_sparse.storage.col()[100:200])

tensor([752, 754, 755, 757, 760, 761, 767, 769, 770, 771, 772, 774, 775, 777,
        778, 781, 783, 785, 786, 788, 789, 790, 791, 792, 793, 796, 797, 798,
        800, 801, 803, 804, 805, 809, 810, 811, 813, 814, 818, 820, 821, 823,
        824, 825, 826, 827, 828, 829, 831, 833, 834, 835, 836, 837, 838, 840,
        841, 843, 844, 846, 847, 849, 850, 852, 855, 856, 857, 858, 862, 864,
        868, 712, 834, 873, 882, 884, 885, 888, 893, 896, 897, 898, 899, 635,
        644, 654, 662, 670, 671, 678, 679, 684, 686, 698, 707, 708, 718, 726,
        737, 751])


## Neural Graph Collaborative Filtering



In [41]:
from torch_geometric.nn import Linear
from torch.nn import Embedding
from torch.nn import functional as F

class EmbeddingPropLayer(torch.nn.Module):
    def __init__(self, hidden_channels=128):
        super(EmbeddingPropLayer, self).__init__()

        self.W1 = Linear(hidden_channels, hidden_channels, bias=False)
        self.W2 = Linear(hidden_channels, hidden_channels, bias=False)

    def reset_parameters(self):
        self.W1.reset_parameters()
        self.W2.reset_parameters()

    def forward(self, E, E_final):
        message = self.message_aggregation(E)
        return message, torch.concat([E_final, message], dim=1)

    def message_construction(self, E, p_ui):
        return torch.mul(p_ui*E, p_ui*self.W2(E))

    def message_aggregation(self, E):
        p_ui = 1
        m_ui = self.message_construction(E, p_ui)
        m_uu = p_ui*self.W1(E)
        return F.leaky_relu(m_uu + m_ui)


class EmbeddingLayer(torch.nn.Module):
    """
        user u and item i with an embedding vector e_u that's an element of d real numbers
        and e_i that's an element of a d real numbers
    """
    def __init__(self, num_of_edges, hidden_channels=128):
        super(EmbeddingLayer, self).__init__()
        self.user_embedding = Embedding(num_of_edges, hidden_channels)
        self.movie_embedding = Embedding(num_of_edges, hidden_channels)

    def forward(self, user_nodes, movie_nodes):
        e_u = self.user_embedding(user_nodes)
        e_i = self.movie_embedding(movie_nodes)
        E = torch.concat([e_u, e_i])

        return E

class NGCF(torch.nn.Module):
    def __init__(self, num_of_users, num_of_movies, hidden_channels=128):
        super(NGCF, self).__init__()
        self.num_of_users = num_of_users
        self.num_of_movies = num_of_movies
        self.num_of_edges = num_of_users + num_of_movies
        self.hidden_channels = hidden_channels

        self.embeddings = EmbeddingLayer(self.num_of_edges, hidden_channels)

        self.embedding_prop_layer_1 = EmbeddingPropLayer(hidden_channels)
        self.embedding_prop_layer_2 = EmbeddingPropLayer(hidden_channels)
        self.embedding_prop_layer_3 = EmbeddingPropLayer(hidden_channels)


    def forward(self, user_nodes, movie_nodes):
        E = self.embeddings(user_nodes, movie_nodes) # size -> [num_users + num_movies, hidden_channels]

        #assert E.size()[0] == self.num_of_edges and E.size()[1] == self.hidden_channels

        E_1, E_star = self.embedding_prop_layer_1(E, torch.empty_like(E)) #E_l -> [num_users+num_movies,
        E_2, E_star = self.embedding_prop_layer_2(E_1, E_star)
        E_3, E_star = self.embedding_prop_layer_2(E_2, E_star)

        #assert E_star.size()[0] == self.num_of_edges and E_star.size()[1] == self.hidden_channels*4

        e_u_star = E_star[:(self.num_of_users-1), :]
        e_i_star = E_star[self.num_of_users:, :]

        # users_emb_final, users_emb_0, items_emb_final, items_emb_0
        return e_u_star, E[:(self.num_of_users-1), :], e_i_star, E_star[self.num_of_users:, :]


In [8]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618
    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for regularization loss term

    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2))

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1)
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1)
    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [9]:
def sample_batch(batch_size, edge_index):
 # returns a tuple of 3 tensors. Tensor 1 -> user, Tensor 2 -> positive interactions, Tensor3-> Neg interactions
   edges = structured_negative_sampling(edge_index)
   edges = torch.stack(edges, dim=0)
   indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
   batch = edges[:, indices]
   user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
   return user_indices, pos_item_indices, neg_item_indices


In [10]:
def get_ground_truth(edge_index):
    """Generates dictionary of positive items for each user efficiently.
    Args:
        edge_index (torch.Tensor): 2 by N list of edges
    Returns:
        dict: dictionary of positive items for each user
    """
    user_pos_items = {user.item(): [] for user in edge_index[0].unique()}
    for user, item in zip(edge_index[0], edge_index[1]):
        user_pos_items[user.item()].append(item.item())

    return user_pos_items

In [11]:
def metrics(groundTruth, r, k):
    num_correct_pred = torch.sum(r, dim=-1).float()
    user_num_liked = torch.tensor([len(groundTruth[i]) for i in range(len(groundTruth))], dtype=torch.float)
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [12]:
# wrapper function to get evaluation metrics
def get_metrics(model, edge_index, exclude_edge_indices, k):
    """Computes the evaluation metrics: recall, precision, and ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on

    Returns:
        tuple: recall @ k, precision @ k, ndcg @ k
    """
    user_embedding = model.users_emb.weight
    item_embedding = model.items_emb.weight

    rating = torch.matmul(user_embedding, item_embedding.T)

    for exclude_edge_index in exclude_edge_indices:
        user_pos_items = get_ground_truth(exclude_edge_index)
        exclude_users = []
        exclude_items = []
        for user, items in user_pos_items.items():
            exclude_users.extend([user] * len(items))
            exclude_items.extend(items)

        rating[exclude_users, exclude_items] = -(1 << 10)

    _, top_K_items = torch.topk(rating, k=k)

    users = edge_index[0].unique()

    test_user_pos_items = get_ground_truth(edge_index)

    test_user_pos_items_list = [
        test_user_pos_items[user.item()] for user in users]

    r = []
    for user in users:
        ground_truth_items = test_user_pos_items[user.item()]
        label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
        r.append(label)
    r = torch.Tensor(np.array(r).astype('float'))

    recall, precision = metrics(test_user_pos_items_list, r, k)
    # ndcg =

    return recall, precision, 0

In [38]:
def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
    """Evaluates model loss and metrics including recall, precision, ndcg @ k

    Args:
        model (LighGCN): lightgcn model
        edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
        sparse_edge_index (sparseTensor): sparse adjacency matrix for split to evaluate
        exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
        k (int): determines the top k items to compute metrics on
        lambda_val (float): determines lambda for bpr loss

    Returns:
        tuple: bpr loss, recall @ k, precision @ k, ndcg @ k
    """
    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
        sparse_edge_index.storage.row(), sparse_edge_index.storage.col())
    edges = structured_negative_sampling(
        edge_index, contains_neg_self_loops=False)
    user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
        pos_item_indices], items_emb_0[pos_item_indices]
    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
        neg_item_indices], items_emb_0[neg_item_indices]

    loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0,
                    neg_items_emb_final, neg_items_emb_0, lambda_val).item()

    recall, precision, ndcg = get_metrics(
        model, edge_index, exclude_edge_indices, k)

    return loss, recall, precision, ndcg

In [14]:
ITERATIONS = 10000
BATCH_SIZE = 1024
LR = 1e-3
ITERS_PER_EVAL = 200
ITERS_PER_LR_DECAY = 200
K = 20
LAMBDA = 1e-6

In [46]:
model = NGCF(len(train_sparse.storage.row()), len(train_sparse.storage.col()), hidden_channels=128)
model.train()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

print(model)

NGCF(
  (embeddings): EmbeddingLayer(
    (user_embedding): Embedding(155456, 128)
    (movie_embedding): Embedding(155456, 128)
  )
  (embedding_prop_layer_1): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
  (embedding_prop_layer_2): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
  (embedding_prop_layer_3): EmbeddingPropLayer(
    (W1): Linear(128, 128, bias=False)
    (W2): Linear(128, 128, bias=False)
  )
)


In [47]:
train_losses = []
val_losses = []

from tqdm import tqdm

for iter in tqdm(range(ITERATIONS)):
   users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
       train_sparse.storage.row(), train_sparse.storage.col())

   user_indices, pos_item_indices, neg_item_indices = sample_batch(
       BATCH_SIZE, train_edge_index)
   user_indices, pos_item_indices, neg_item_indices = user_indices.to(
       device), pos_item_indices.to(device), neg_item_indices.to(device)
   users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
   pos_items_emb_final, pos_items_emb_0 = items_emb_final[
       pos_item_indices], items_emb_0[pos_item_indices]
   neg_items_emb_final, neg_items_emb_0 = items_emb_final[
       neg_item_indices], items_emb_0[neg_item_indices]

   train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
                         pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

   optimizer.zero_grad()
   train_loss.backward()
   optimizer.step()

   if iter % ITERS_PER_EVAL == 0:
       model.eval()
       val_loss, recall, precision, ndcg = evaluation(
           model, val_edge_index, val_sparse, [train_edge_index], K, LAMBDA)
       print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
       train_losses.append(train_loss.item())
       val_losses.append(val_loss)
       model.train()

   if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
       scheduler.step()

  0%|          | 0/10000 [00:01<?, ?it/s]


IndexError: index is out of bounds for dimension with size 0